Re-worked code from PublicRecords_NLP.ipynb to run NLP on site feedback data in an effort to figure out what site users are interested in.

Dataset contains some confidential information and is was not uploaded to a public repo to avoid crawlers. (Included in ZIP)

In [ ]:
# Install necessary libraries
%%capture
!pip install bertopic
import pandas as pd
import spacy
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Feedback on Website
feedback = pd.read_excel('Was This Page Helpful.xlsx',skiprows=2)
feedback = feedback['Feedback'].tolist()
feedback = [str(i) for i in feedback]

/
/content


In [ ]:
# training
topic_model_1 = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model_1.fit_transform(feedback)

Batches:   0%|          | 0/343 [00:00<?, ?it/s]

In [ ]:
# save model 1
topic_model_1.save("Site Feedback")	

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
# most frequent topics: -1 indicates outliers, ignore
freq = topic_model_1.get_topic_info(); freq.head(20)

,Topic,Count,Name
0,-1,2940,-1_the_to_in_and
1,0,320,0_tax_taxes_property_sales
2,1,299,1_trash_recycling_pickup_pick
3,2,264,2_application_apply_easy_process
4,3,162,3_email_contact_address_addresses
5,4,159,4_map_maps_zoning_district
6,5,153,5_report_police_accident_file
7,6,145,6_ticket_pay_tickets_number
8,7,131,7_updated_update_last_needs
9,8,128,8_monuments_history_statues_confederate


In [ ]:
topic_model_1.get_topic(0) # get the most frequent topic

[('tax', 0.050199305159967546),
 ('taxes', 0.026783154282595076),
 ('property', 0.018971400950171514),
 ('sales', 0.01877661965370169),
 ('inheritance', 0.018456244377285824),
 ('pay', 0.014763474295859622),
 ('estate', 0.014328734659282584),
 ('rolls', 0.012025673091779534),
 ('bill', 0.011481949881338821),
 ('2020', 0.009079972168247434)]

In [ ]:
topic_model_1.get_topic(1) # get the 2nd most frequent topic

[('trash', 0.040353702914277645),
 ('recycling', 0.03747018069131202),
 ('pickup', 0.019263657385025203),
 ('pick', 0.018285225919266734),
 ('garbage', 0.018239588169170335),
 ('recycle', 0.014477882738934276),
 ('picked', 0.012048832975068079),
 ('waste', 0.011837504000493285),
 ('glass', 0.011759256661818257),
 ('up', 0.011055855668862715)]

In [ ]:
# size of each circle = frequency of topic
interactive_map = topic_model_1.visualize_topics(); interactive_map

In [ ]:
# Set labels for top 10 topics
topic_model_1.set_topic_labels({0: "Taxes", 1: "Trash and Recycling",
                                2: "Easy Application Process", 3: "Emails", 4: "Location Data",
                                5: "Police Report", 6: "Ticket Payments", 7: "Needs Update",
                                8: "Monument Removal", 9: "Easy Page Navigation"})

# Display model 1 bar chart
topic_model_1.visualize_barchart(top_n_topics=10, title='Model 1: Topic Word Scores',
                                 custom_labels=True)

In [ ]:
# change ngram_range, let each entity in a topic be up to 3 words
vectorizer_model = CountVectorizer(ngram_range=(1, 3))

# update model 1
topic_model_1.update_topics(feedback, vectorizer_model=vectorizer_model)

In [ ]:
# get most frequent topics: ngram_range = (1,3)
freq = topic_model_1.get_topic_info(); freq.head(20)

,Topic,Count,Name,CustomName
0,-1,2940,-1_the_to_and_in,-1_the_to_in_and
1,0,320,0_tax_taxes_property_sales,Taxes
2,1,299,1_trash_recycling_pickup_pick,Trash and Recycling
3,2,264,2_application_the application_apply_to apply,Easy Application Process
4,3,162,3_email_contact_email address_address,Emails
5,4,159,4_map_the map_maps_district,Location Data
6,5,153,5_report_police report_police_accident,Police Report
7,6,145,6_ticket_pay_my ticket_to pay,Ticket Payments
8,7,131,7_updated_update_be updated_not updated,Needs Update
9,8,128,8_history_monuments_statues_confederate,Monument Removal


In [ ]:
# train a new model with modified topic sizes (min topic size = 5)
topic_model_2 = BERTopic(language="english", min_topic_size=5, calculate_probabilities=True, verbose=True)
topics, probs = topic_model_2.fit_transform(feedback)

Batches:   0%|          | 0/343 [00:00<?, ?it/s]

In [ ]:
# save model 2
topic_model_2.save("Site Feedback 2")	

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
# most frequent topics: model 2
freq = topic_model_2.get_topic_info(); freq.head(20)

,Topic,Count,Name
0,-1,3150,-1_in_to_the_and
1,0,173,0_email_contact_address_addresses
2,1,155,1_ticket_tickets_pay_system
3,2,128,2_recycling_recycle_hazardous_glass
4,3,118,3_monuments_statues_history_confederate
5,4,114,4_permit_permits_status_utilityservice
6,5,110,5_nola_nolagov_testing_red
7,6,106,6_staff_department_departments_director
8,7,99,7_trash_pick_pickup_garbage
9,8,86,8_occupational_license_renew_renewal


In [ ]:
topic_model_2.get_topic(0) # get the most frequent topic

[('email', 0.054527725079008896),
 ('contact', 0.039020025785484753),
 ('address', 0.028607607741412248),
 ('addresses', 0.022244878881574795),
 ('send', 0.015626897566013116),
 ('emails', 0.01234298494628762),
 ('no', 0.0095825409662155),
 ('gmailcom', 0.008937100712796851),
 ('an', 0.008597183646683378),
 ('contacts', 0.008508359200230776)]

In [ ]:
topic_model_2.get_topic(1) # get the 2nd most frequent topic

[('ticket', 0.03147025857097084),
 ('tickets', 0.015533266276818862),
 ('pay', 0.015488413544375008),
 ('system', 0.013660472576863222),
 ('number', 0.010375101693643932),
 ('online', 0.008915245453364461),
 ('seatbelt', 0.00784513403900821),
 ('lost', 0.007205221571427408),
 ('my', 0.006523218579567387),
 ('traffic', 0.006021621630494656)]

In [ ]:
# Set labels for top 10 topics
topic_model_2.set_topic_labels({0: "Emails", 1: "Ticket Payments",
                                2: "Recycling", 3: "Monument Removal", 4: "Permits",
                                5: "Site Problems", 6: "Government Staff Info", 7: "Trash",
                                8: "License Renewal", 9: "Meeting Scheduling"})

# Display model 2 bar chart
topic_model_2.visualize_barchart(top_n_topics=10, title='Model 2: Topic Word Scores',
                                 custom_labels=True)

In [ ]:
# train a new model with modified topic sizes (topic size = 5)
topic_model_3 = BERTopic(language="english", top_n_words=5, min_topic_size=5,
                         calculate_probabilities=True, verbose=True)
topics, probs = topic_model_3.fit_transform(feedback)

Batches:   0%|          | 0/343 [00:00<?, ?it/s]

In [ ]:
# save model 3
topic_model_3.save("Site Feedback 3")	

In [ ]:
# most frequent topics: model 3
freq = topic_model_3.get_topic_info(); freq.head(20)

,Topic,Count,Name
0,-1,3124,-1_in_the_to_and
1,0,145,0_ticket_pay_tickets_system
2,1,122,1_monuments_statues_history_confederate
3,2,120,2_staff_department_departments_director
4,3,105,3_permit_permits_status_utilityservice
5,4,104,4_nola_nolagov_testing_red
6,5,92,5_occupational_license_renew_renewal
7,6,85,6_helpful_very_helpfull_helpfully
8,7,85,7_parking_handicapped_signs_park
9,8,82,8_job_jobs_looking_position


In [ ]:
topic_model_3.get_topic(0) # get the most frequent topic

[('ticket', 0.032977240241493434),
 ('pay', 0.01586835317611595),
 ('tickets', 0.015111696787445207),
 ('system', 0.012852234187100514),
 ('seatbelt', 0.011685437239165986)]

In [ ]:
topic_model_3.get_topic(1) # get the 2nd most frequent topic

[('monuments', 0.019571791629958795),
 ('statues', 0.017965412810346994),
 ('history', 0.016305375449431267),
 ('confederate', 0.015479648191900125),
 ('removal', 0.010304591552296882)]

In [ ]:
# Set labels for top 10 topics
topic_model_3.set_topic_labels({0: "Ticket Payment", 1: "Monument Removal",
                                2: "Government Staff Info", 3: "Permits", 4: "Site Problems",
                                5: "License Renewal", 6: "Positive reviews", 7: "Parking",
                                8: "Jobs", 9: "Parking Tickets"})

# Display model 3 bar chart
topic_model_3.visualize_barchart(top_n_topics=10, title='Model 3: Topic Word Scores',
                                 custom_labels=True)

In [ ]:
# train new model, add diversity
topic_model_4 = BERTopic(language="english", calculate_probabilities=True, diversity=0.5, verbose=True)
topics, probs = topic_model_4.fit_transform(feedback)

Batches:   0%|          | 0/343 [00:00<?, ?it/s]

In [ ]:
# save model 4
topic_model_4.save("Site Feedback 4")

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
# most frequent topics: model 4
freq = topic_model_4.get_topic_info(); freq.head(20)

,Topic,Count,Name
0,-1,2750,-1_of_to_act_city
1,0,339,0_trash_recycling_schedule_hazardous
2,1,256,1_application_process_assistance_submit
3,2,235,2_ticket_pay_number_camera
4,3,233,3_permits_building_status_need
5,4,174,4_tax_estate_bill_real
6,5,170,5_occupational_renewal_online_form
7,6,165,6_saints_orleans_retrieved_nfl
8,7,158,7_zoning_maps_districts_historic
9,8,158,8_contact_addresses_mail_contacts


In [ ]:
topic_model_4.get_topic(0) # get the most frequent topic

[('trash', 0.03532591746485051),
 ('recycling', 0.033491435840022475),
 ('schedule', 0.009247736879856154),
 ('hazardous', 0.008842625121982153),
 ('disposal', 0.0072081101662160914),
 ('bin', 0.006875363827770731),
 ('drop', 0.006645283329274012),
 ('debris', 0.006318881854952033),
 ('cans', 0.006259780483947966),
 ('plastic', 0.006133736383234737)]

In [ ]:
topic_model_4.get_topic(1) # get the 2nd most frequent topic

[('application', 0.06729468986999905),
 ('process', 0.013548284340057042),
 ('assistance', 0.013545391406992931),
 ('submit', 0.013408886204651468),
 ('app', 0.012273226834474309),
 ('fill', 0.011807524329983408),
 ('very', 0.008199609742785524),
 ('approved', 0.00788290775257908),
 ('status', 0.006928980341811505),
 ('applying', 0.0061565374039563026)]

In [ ]:
# Set labels for top 10 topics
topic_model_4.set_topic_labels({0: "Waste Disposal", 1: "Application Process",
                                2: "Ticket Payment", 3: "Building Permit", 4: "Tax and Bill Payments",
                                5: "License Renewal", 6: "Saints (NFL)", 7: "Location Info",
                                8: "Identifying Info", 9: "Incident Reports"})

# Display model 4 bar chart
topic_model_4.visualize_barchart(top_n_topics=10, title='Model 4: Topic Word Scores',
                                 custom_labels=True)